# 🎓 Agent Vocal IA - Setup Colab

Notebook d'installation et de vérification pour l'Agent Vocal IA éducatif 100% local.

**Fonctionnalités :**
- 🎤 ASR (Faster-Whisper + Silero VAD)
- 🔍 RAG (FAISS + SentenceTransformers)
- 🧠 LLM local (llama-cpp-python)
- 🔊 TTS (gTTS/pyttsx3 - Python 3.12 compatible)
- 💡 Système de hints progressifs

**Prérequis :** GPU T4 ou A100 recommandé

---

⚠️ **IMPORTANT** : Activez le GPU avant de commencer !
- Menu **Runtime** → **Change runtime type** → **T4 GPU** → **Save**

## 📋 Étape 1 : Vérification de l'environnement

Vérifions d'abord que nous avons bien accès à un GPU.

In [ ]:
# Vérification GPU et CUDA
import subprocess
import sys
import platform

print("=" * 60)
print("🖥️  INFORMATIONS SYSTÈME")
print("=" * 60)
print(f"Python version: {sys.version}")
print(f"Plateforme: {platform.platform()}")
print()

# Vérifier NVIDIA GPU
print("=" * 60)
print("🎮  VÉRIFICATION GPU")
print("=" * 60)
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print(result.stdout)
    print("✅ GPU NVIDIA détecté!")
except Exception as e:
    print(f"⚠️  Aucun GPU détecté. L'exécution sera très lente sur CPU.")
    print(f"   Activez le GPU: Runtime → Change runtime type → T4 GPU")

print("=" * 60)

## 📥 Étape 2 : Clonage du projet

Téléchargeons le projet depuis GitHub.

In [ ]:
import os

# Retourner à /content si dans un sous-dossier
if '/content/' in os.getcwd() and os.getcwd() != '/content':
    %cd /content

# Nettoyer ancien dossier
if os.path.exists('agent_vocal_IA'):
    print("🗑️  Suppression ancienne installation...")
    !rm -rf agent_vocal_IA

# Cloner
print("📥 Clonage...")
!git clone -q https://github.com/Romainmlt123/agent_vocal_IA.git
%cd agent_vocal_IA

print(f"✅ Projet cloné: {os.getcwd()}")

## 📦 Étape 3 : Installation des dépendances

Installation de toutes les bibliothèques requises. **Cela peut prendre 5-10 minutes.**

### ⚠️ Compatibilité Python 3.12+

Google Colab utilise Python 3.12, ce qui pose des problèmes :

- **❌ Piper-TTS** : Non compatible Python 3.12+
- **❌ Coqui TTS** : Non compatible Python 3.12+
- **✅ gTTS** : Compatible (nécessite internet)
- **✅ pyttsx3** : Compatible (TTS système)

### 🔧 Ce que fait la cellule suivante

1. Installe **PortAudio** (résout erreur sounddevice)
2. Gère les conflits **NumPy** (force <2.0)
3. FAISS-GPU → fallback FAISS-CPU
4. llama-cpp-python avec CUDA
5. gTTS + pyttsx3 pour TTS

In [ ]:
import sys, subprocess
print("🔄 Installation...\n")

# 1. PortAudio
print("1/8 PortAudio...")
!apt-get update -qq > /dev/null 2>&1
!apt-get install -qq -y portaudio19-dev > /dev/null 2>&1

# 2. Désinstaller packages incompatibles NumPy 2.0
print("2/8 Nettoyage environnement...")
!pip uninstall -y -q opencv-python opencv-python-headless opencv-contrib-python jax jaxlib pytensor thinc 2>/dev/null

# 3. NumPy <2.0 (CRITIQUE pour sentence-transformers)
print("3/8 NumPy <2.0...")
!pip install --force-reinstall -q 'numpy>=1.24,<2.0'

# 4. Réinstaller opencv compatible
print("4/8 OpenCV compatible...")
!pip install -q 'opencv-python-headless>=4.5,<4.10'

# 5. Packages base
print("5/8 Packages base...")
!pip install -q torch torchaudio faster-whisper==1.0.3 sentence-transformers==2.7.0 pypdf==4.2.0 langchain==0.2.11 langchain-community==0.2.10 gradio>=4.44.0 pyyaml>=6.0.2 soundfile sounddevice

# 6. FAISS
print("6/8 FAISS...")
if subprocess.run(['pip','install','-q','faiss-gpu'], capture_output=True).returncode != 0:
    !pip install -q faiss-cpu==1.8.0

# 7. Silero VAD
print("7/8 Silero VAD...")
!pip install -q silero-vad

# 8. llama-cpp-python
print("8/8 llama-cpp-python...")
if subprocess.run(['pip','install','-q','llama-cpp-python','--extra-index-url','https://abetlen.github.io/llama-cpp-python/whl/cu121'], capture_output=True).returncode != 0:
    !pip install -q llama-cpp-python

# 9. TTS
print("9/9 TTS...")
!pip install -q gTTS pyttsx3

print("\n✅ Installation terminée!")
print("📝 TTS: gTTS installé (Piper incompatible Python 3.12+)")

## ✅ Étape 4 : Vérification des imports

Testons que tous les modules principaux peuvent être importés.

In [ ]:
# ⚠️  IMPORTANT: Redémarrer le kernel après installation NumPy
print("⚠️  REDÉMARRAGE KERNEL...")
print("   Nécessaire après réinstallation NumPy\n")

import os
os.kill(os.getpid(), 9)  # Force restart

In [ ]:
# Test imports après restart
import sys

def test(module, name=None):
    name = name or module
    try:
        __import__(module.split('.')[0])
        print(f"✅ {name}")
        return True
    except:
        print(f"❌ {name}")
        return False

print("📚 Vérification...\n")
r = [
    test('torch'),
    test('faster_whisper'),
    test('sentence_transformers'),
    test('faiss'),
    test('langchain'),
    test('llama_cpp'),
    test('gradio'),
    test('numpy'),
    test('sounddevice'),
    test('gtts')
]

print(f"\n📊 {sum(r)}/{len(r)} modules OK")
if sum(r) == len(r):
    print("🎉 Prêt!")
else:
    print("⚠️  Certains modules manquent")

## 📥 Étape 5 : Téléchargement des modèles

Téléchargement des modèles LLM. **Durée : 10-15 minutes**

In [ ]:
import os
import urllib.request
from pathlib import Path

def download_file(url, destination, description="Fichier"):
    print(f"📥 {description}...")
    print(f"   → {destination}")
    
    os.makedirs(os.path.dirname(destination), exist_ok=True)
    
    if os.path.exists(destination):
        print(f"   ✅ Déjà présent")
        return True
    
    try:
        def progress(block, block_size, total):
            percent = min(100, block * block_size * 100 / total) if total > 0 else 0
            bar = '█' * int(percent / 2.5) + '░' * (40 - int(percent / 2.5))
            print(f'\r   [{bar}] {percent:.1f}%', end='', flush=True)
        
        urllib.request.urlretrieve(url, destination, progress)
        print(f"\n   ✅ Téléchargé!")
        return True
    except Exception as e:
        print(f"\n   ❌ Erreur: {e}")
        return False

print("=" * 60)
print("📦 TÉLÉCHARGEMENT DES MODÈLES")
print("=" * 60)

Path("models/llm").mkdir(parents=True, exist_ok=True)

print("\n🦙 Modèle LLM (Phi-3 Mini - 2.4 GB)")
llm_url = "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf"
llm_dest = "models/llm/phi-3-mini-4k-instruct.Q4_K_M.gguf"
download_file(llm_url, llm_dest, "Phi-3 Mini 4K")

print("\n" + "=" * 60)
print("🎉 Téléchargement terminé!")
print("=" * 60)

## 🧪 Étape 6 : Construction des indices RAG

Construisons les indices pour les 3 matières.

In [ ]:
print("=" * 60)
print("🔍 CONSTRUCTION DES INDICES RAG")
print("=" * 60)

print("\n📐 Mathématiques...")
!python -m src.rag_build --subject maths

print("\n⚡ Physique...")
!python -m src.rag_build --subject physique

print("\n🇬🇧 Anglais...")
!python -m src.rag_build --subject anglais

print("\n=" * 60)
print("✅ Indices RAG construits!")
print("=" * 60)

## 🚀 Étape 7 : Lancement de l'interface

Lançons l'interface Gradio avec lien public.

In [ ]:
print("🚀 Lancement de l'interface Gradio...")
print("=" * 60)

from ui.app import launch_ui

# Lancer avec lien public
launch_ui(share=True)

print("\n📝 Cliquez sur le lien ci-dessus pour accéder à l'interface")
print("⚠️  Le micro ne fonctionne pas dans l'iframe Colab, utilisez le lien public!")

## 🎉 Installation terminée !

Votre Agent Vocal IA est prêt. Cliquez sur le lien Gradio ci-dessus.

### ⚠️ Note importante sur TTS

Python 3.12+ n'est pas compatible avec Piper-TTS. Vous avez 2 options :

1. **Utiliser le mode Texte** (pas de synthèse vocale)
2. **Adapter le code** pour utiliser gTTS (nécessite modification de `src/tts.py`)

Pour les tests, le mode texte fonctionne parfaitement !

# 🎓 Agent Vocal IA - Setup Colab

Notebook d'installation et de vérification pour l'Agent Vocal IA éducatif 100% local.

**Fonctionnalités :**
- 🎤 ASR (Faster-Whisper + Silero VAD)
- 🔍 RAG (FAISS + SentenceTransformers)
- 🧠 LLM local (llama-cpp-python)
- 🔊 TTS (Piper-TTS)
- 💡 Système de hints progressifs

**Prérequis :** GPU T4 ou A100 recommandé

## 📋 Étape 1 : Vérification de l'environnement

Vérifions d'abord que nous avons bien accès à un GPU et les informations système.

In [1]:
# Vérification GPU et CUDA
import subprocess
import sys
import platform

print("=" * 60)
print("🖥️  INFORMATIONS SYSTÈME")
print("=" * 60)
print(f"Python version: {sys.version}")
print(f"Plateforme: {platform.platform()}")
print(f"Architecture: {platform.machine()}")
print()

# Vérifier NVIDIA GPU
print("=" * 60)
print("🎮  VÉRIFICATION GPU")
print("=" * 60)
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print(result.stdout)
except Exception as e:
    print(f"⚠️  Aucun GPU NVIDIA détecté ou nvidia-smi non disponible: {e}")
    print("Note: L'exécution sera possible sur CPU mais beaucoup plus lente.")

print()
print("=" * 60)
print("📦  VÉRIFICATION PYTORCH")
print("=" * 60)
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"Nombre de GPUs: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"    Mémoire totale: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
except ImportError:
    print("PyTorch n'est pas encore installé. Il sera installé à l'étape suivante.")
print("=" * 60)

🖥️  INFORMATIONS SYSTÈME
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Plateforme: Linux-6.6.105+-x86_64-with-glibc2.35
Architecture: x86_64

🎮  VÉRIFICATION GPU
Wed Oct 29 20:07:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |      

## 📥 Étape 2 : Clonage du dépôt (si nécessaire)

Si vous n'avez pas encore cloné le dépôt, décommentez et exécutez la cellule suivante.

In [2]:
# Décommentez les lignes suivantes si vous devez cloner le dépôt
!git clone https://github.com/Romainmlt123/agent_vocal_IA.git
%cd agent_vocal_IA

# Si vous êtes déjà dans le dossier, vérifiez la structure
import os
print("📁 Dossier actuel:", os.getcwd())
print("\n📂 Structure du projet:")
for root, dirs, files in os.walk('.', topdown=True):
    # Limiter la profondeur pour la lisibilité
    level = root.replace('.', '').count(os.sep)
    if level < 3:
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            if not file.startswith('.'):
                print(f'{subindent}{file}')

Cloning into 'agent_vocal_IA'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 117 (delta 18), reused 106 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 240.99 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/agent_vocal_IA
📁 Dossier actuel: /content/agent_vocal_IA

📂 Structure du projet:
./
  CHANGELOG.md
  Cahier des charges - Intelligence Lab - Romain Mallet (1).pdf
  README.md
  demo_cli.py
  LICENSE
  requirements.txt
  setup_colab.ipynb
  test_conversation.py
  config.yaml
  src/
    orchestrator.py
    rag_build.py
    llm.py
    rag.py
    utils.py
    __init__.py
    tts.py
    conversation_manager.py
    asr.py
  ui/
    __init__.py
    app.py
  data/
    maths/
      cours_maths.md
    physique/
      cours_physique.md
    anglais/
      english_grammar.md
  tests/
    test_utils.py
    __init__.py
    test_integration.py
    t

## 📦 Étape 3 : Installation des dépendances

Installation de toutes les bibliothèques requises. **Cela peut prendre 5-10 minutes.**

### ⚠️ Compatibilité Python 3.12+

Google Colab utilise Python 3.12, ce qui pose des problèmes avec certaines bibliothèques :

- **❌ Piper-TTS** : Non compatible Python 3.12+ (seulement 3.9-3.11)
- **❌ Coqui TTS** : Non compatible Python 3.12+ (seulement 3.9-3.11)
- **✅ gTTS** : Compatible, mais nécessite internet
- **✅ pyttsx3** : Compatible, TTS système

### 🔧 Solutions Automatiques

La cellule suivante :
1. Installe **PortAudio** (requis pour sounddevice)
2. Gère les conflits **NumPy** (force <2.0 pour compatibilité)
3. Essaie **FAISS-GPU** → fallback **FAISS-CPU**
4. Installe **llama-cpp-python** avec wheels précompilés CUDA
5. Pour TTS :
   - Python 3.12+ → installe **gTTS + pyttsx3**
   - Python 3.10/3.11 → essaie **Piper-TTS**, sinon gTTS

**Note** : Si vous avez besoin de Piper-TTS, vous devrez utiliser un runtime Python 3.10 ou 3.11.

In [ ]:
# Installation des dépendances - Version optimisée pour Colab
print("🔄 Installation des packages...")
print("=" * 60)

# Vérifier la version de Python
import sys
print(f"📌 Python version: {sys.version}")

# Étape 1 : Installer les dépendances de base sans les packages problématiques
print("\n📦 Étape 1/5 : Installation des packages de base...")
!pip install -q torch>=2.0.0 torchaudio>=2.0.0
!pip install -q faster-whisper==1.0.3
!pip install -q sentence-transformers==2.7.0
!pip install -q pypdf==4.2.0
!pip install -q langchain==0.2.11 langchain-community==0.2.10
!pip install -q gradio==4.36.1
!pip install -q pyyaml==6.0.1
!pip install -q numpy scipy soundfile sounddevice

# Étape 2 : Installer FAISS (essayer GPU d'abord, puis CPU en fallback)
print("\n📊 Étape 2/5 : Installation de FAISS...")
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True, text=True)
if result.returncode == 0:
    print("✅ FAISS-GPU installé avec succès!")
else:
    print("⚠️  FAISS-GPU non disponible, installation de FAISS-CPU...")
    !pip install -q faiss-cpu==1.8.0

# Étape 3 : Installer Silero VAD
print("\n🎙️  Étape 3/5 : Installation de Silero VAD...")
!pip install -q silero-vad

# Étape 4 : Installer llama-cpp-python (version précompilée)
print("\n🦙 Étape 4/5 : Installation de llama-cpp-python...")
print("   Tentative avec version CUDA précompilée...")
result = subprocess.run(['pip', 'install', '-q', 'llama-cpp-python', '--extra-index-url', 'https://abetlen.github.io/llama-cpp-python/whl/cu121'], capture_output=True, text=True)
if result.returncode == 0:
    print("✅ llama-cpp-python avec CUDA installé!")
else:
    print("⚠️  Version CUDA échouée, installation version standard...")
    !pip install -q llama-cpp-python==0.2.85

# Étape 5 : TTS - Utiliser une alternative compatible
print("\n🔊 Étape 5/5 : Configuration TTS...")
print("⚠️  Note: piper-tts n'est pas compatible avec Python 3.12+")
print("   Installation de Coqui TTS comme alternative compatible...")
!pip install -q TTS>=0.22.0

print("\n" + "=" * 60)
print("✅ Installation terminée!")
print("=" * 60)
print("\n📝 Notes importantes:")
print("   • Si llama-cpp-python a échoué, le LLM utilisera le CPU")
print("   • Pour TTS, nous utilisons Coqui TTS au lieu de Piper")
print("   • Vérifiez les imports dans la cellule suivante")

🔄 Installation des packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.0 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 0.5.12 Requires-Python >=3.7,<3.12; 0.5.13 Requires-Python >=3.7,<3.12; 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement piper-phonemize~=1.1.0 (from piper-tts) (from versions: none)
ERROR: No matching distribution found for piper-phonemize~=1.1.0

📊 Installation de FAISS-GPU p

## ✅ Étape 4 : Vérification des imports

Testons que tous les modules principaux peuvent être importés correctement.

In [ ]:
# Test des imports critiques
import sys

def test_import(module_name, display_name=None, alternative_names=None):
    """Test l'import d'un module et affiche le résultat."""
    if display_name is None:
        display_name = module_name
    
    # Essayer le module principal
    try:
        module = __import__(module_name.split('.')[0])
        version = getattr(module, '__version__', 'version inconnue')
        print(f"✅ {display_name:30} - {version}")
        return True
    except ImportError as e:
        # Essayer les alternatives si fournies
        if alternative_names:
            for alt_name in alternative_names:
                try:
                    module = __import__(alt_name.split('.')[0])
                    version = getattr(module, '__version__', 'version inconnue')
                    print(f"✅ {display_name:30} - {version} (via {alt_name})")
                    return True
                except ImportError:
                    continue
        
        print(f"❌ {display_name:30} - NON DISPONIBLE")
        return False
    except Exception as e:
        print(f"⚠️  {display_name:30} - ERREUR: {str(e)[:50]}")
        return False

print("=" * 60)
print("📚 VÉRIFICATION DES BIBLIOTHÈQUES")
print("=" * 60)

modules_to_test = [
    ('torch', 'PyTorch', None),
    ('torchaudio', 'TorchAudio', None),
    ('faster_whisper', 'Faster-Whisper (ASR)', None),
    ('silero_vad', 'Silero VAD', None),
    ('sentence_transformers', 'SentenceTransformers', None),
    ('faiss', 'FAISS', None),
    ('pypdf', 'PyPDF', None),
    ('langchain', 'LangChain', None),
    ('llama_cpp', 'llama-cpp-python', None),
    ('gradio', 'Gradio', None),
    ('yaml', 'PyYAML', None),
    ('numpy', 'NumPy', None),
    ('soundfile', 'SoundFile', None),
    ('sounddevice', 'SoundDevice', None),
]

# Test TTS selon ce qui est installé
python_version = sys.version_info
if python_version.minor >= 12:
    modules_to_test.append(('gtts', 'gTTS (TTS fallback)', ['pyttsx3']))
else:
    modules_to_test.append(('piper', 'Piper-TTS', ['gtts', 'TTS']))

results = []
for module_info in modules_to_test:
    if len(module_info) == 3:
        module, name, alts = module_info
        results.append(test_import(module, name, alts))
    else:
        module, name = module_info
        results.append(test_import(module, name))

print("=" * 60)
success_count = sum(results)
total_count = len(results)
print(f"\n📊 Résultat: {success_count}/{total_count} modules importés avec succès")

if success_count == total_count:
    print("🎉 Tous les modules sont prêts!")
elif success_count >= total_count - 2:
    print("✅ La plupart des modules sont prêts!")
    print("💡 Les modules manquants ne sont pas critiques pour le fonctionnement de base.")
else:
    print("⚠️  Plusieurs modules ont échoué. Vérifiez les erreurs ci-dessus.")
    print("💡 Réexécutez la cellule d'installation si nécessaire.")

# Test spécial pour sounddevice (PortAudio)
print("\n" + "=" * 60)
print("🎵 TEST SPÉCIAL: SoundDevice/PortAudio")
print("=" * 60)
try:
    import sounddevice as sd
    devices = sd.query_devices()
    print(f"✅ SoundDevice opérationnel - {len(devices)} périphériques audio détectés")
except Exception as e:
    print(f"⚠️  SoundDevice erreur: {str(e)[:100]}")
    print("   Note: Normal sur Colab (pas de périphériques audio virtuels)")
print("=" * 60)

## 📥 Étape 5 : Téléchargement des modèles

Téléchargement des modèles LLM et voix TTS. **Attention: peut prendre 10-15 minutes selon la connexion.**

In [ ]:
import os
import urllib.request
from pathlib import Path

def download_file(url, destination, description="Fichier"):
    """Télécharge un fichier avec barre de progression."""
    print(f"📥 Téléchargement de {description}...")
    print(f"   URL: {url}")
    print(f"   Destination: {destination}")

    os.makedirs(os.path.dirname(destination), exist_ok=True)

    if os.path.exists(destination):
        print(f"   ✅ Fichier déjà présent, téléchargement ignoré.")
        return True

    try:
        def progress_hook(block_num, block_size, total_size):
            downloaded = block_num * block_size
            percent = min(100, downloaded * 100 / total_size) if total_size > 0 else 0
            bar_length = 40
            filled = int(bar_length * percent / 100)
            bar = '█' * filled + '░' * (bar_length - filled)
            print(f'\r   [{bar}] {percent:.1f}%', end='', flush=True)

        urllib.request.urlretrieve(url, destination, progress_hook)
        print(f"\n   ✅ Téléchargement réussi!")
        return True
    except Exception as e:
        print(f"\n   ❌ Erreur: {e}")
        return False

print("=" * 60)
print("📦 TÉLÉCHARGEMENT DES MODÈLES")
print("=" * 60)

# Créer les dossiers nécessaires
Path("models/llm").mkdir(parents=True, exist_ok=True)
Path("models/voices").mkdir(parents=True, exist_ok=True)

print("\n🦙 Modèle LLM (Phi-3 Mini 4K Instruct - Q4_K_M)")
print("-" * 60)
llm_url = "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf"
llm_dest = "models/llm/phi-3-mini-4k-instruct.Q4_K_M.gguf"
download_file(llm_url, llm_dest, "Phi-3 Mini 4K (2.4 GB)")

print("\n\n🔊 Modèle TTS (Piper - Voix française Siwis)")
print("-" * 60)
tts_model_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/fr/fr_FR/siwis/medium/fr_FR-siwis-medium.onnx"
tts_config_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/fr/fr_FR/siwis/medium/fr_FR-siwis-medium.onnx.json"
tts_model_dest = "models/voices/fr_FR-siwis-medium.onnx"
tts_config_dest = "models/voices/fr_FR-siwis-medium.onnx.json"

download_file(tts_model_url, tts_model_dest, "Voix TTS française (60 MB)")
download_file(tts_config_url, tts_config_dest, "Config TTS")

print("\n" + "=" * 60)
print("🎉 Téléchargement des modèles terminé!")
print("=" * 60)

### ⚠️ Note CRITIQUE sur TTS et Python 3.12+

**PROBLÈME** : Ni Piper-TTS ni Coqui TTS ne fonctionnent avec Python 3.12+.

**Ce que nous avons installé** : gTTS + pyttsx3 (alternatives)

**3 SOLUTIONS POSSIBLES** :

#### Solution 1 : Utiliser gTTS (Recommandé pour tests)
```python
from gtts import gTTS
tts = gTTS("Bonjour", lang='fr')
tts.save("output.mp3")
```
**✅ Avantage** : Simple, fonctionne immédiatement  
**❌ Inconvénient** : Nécessite internet, qualité moyenne

#### Solution 2 : Modifier le code du projet
Vous devez adapter `src/tts.py` pour utiliser gTTS au lieu de Piper.

#### Solution 3 : Downgrade Python (Solution propre)
```python
# ⚠️  ATTENTION: Cela va redémarrer le runtime !
# Décommentez les lignes suivantes pour downgrade vers Python 3.10
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
# !sudo update-alternatives --config python3
# import os
# os.kill(os.getpid(), 9)  # Force restart
```

**RECOMMANDATION** : Pour l'instant, continuez avec gTTS. Les modèles Piper téléchargés ci-dessus resteront disponibles si vous downgrade Python plus tard.

## 🧪 Étape 6 : Test rapide des modules

Testons rapidement chaque composant principal du système.

In [ ]:
# Test rapide de chaque composant
print("=" * 60)
print("🧪 TESTS DES COMPOSANTS")
print("=" * 60)

# Test 1: Configuration
print("\n1️⃣  Test de chargement de la configuration...")
try:
    import yaml
    with open('config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print(f"✅ Configuration chargée: {len(config)} sections")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 2: Faster-Whisper (ASR)
print("\n2️⃣  Test Faster-Whisper (ASR)...")
try:
    from faster_whisper import WhisperModel
    # Ne pas charger le modèle complet, juste vérifier l'import
    print("✅ Faster-Whisper disponible")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 3: SentenceTransformers (Embeddings)
print("\n3️⃣  Test SentenceTransformers (Embeddings)...")
try:
    from sentence_transformers import SentenceTransformer
    # Test d'embedding simple
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding = model.encode("Test de phrase")
    print(f"✅ Embeddings générés: dimension {len(embedding)}")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 4: FAISS
print("\n4️⃣  Test FAISS (Index vectoriel)...")
try:
    import faiss
    import numpy as np
    # Créer un petit index de test
    dimension = 384
    index = faiss.IndexFlatL2(dimension)
    vectors = np.random.random((10, dimension)).astype('float32')
    index.add(vectors)
    print(f"✅ FAISS opérationnel: {index.ntotal} vecteurs indexés")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 5: llama-cpp-python
print("\n5️⃣  Test llama-cpp-python (LLM)...")
try:
    from llama_cpp import Llama
    print("✅ llama-cpp-python disponible")
    # Vérifier si CUDA est supporté
    import llama_cpp
    print(f"   Version: {llama_cpp.__version__ if hasattr(llama_cpp, '__version__') else 'N/A'}")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 6: Gradio
print("\n6️⃣  Test Gradio (UI)...")
try:
    import gradio as gr
    print(f"✅ Gradio {gr.__version__} disponible")
except Exception as e:
    print(f"❌ Erreur: {e}")

print("\n" + "=" * 60)
print("✅ Tests de base terminés!")
print("=" * 60)
print("\n💡 Conseil: Vous pouvez maintenant utiliser les modules du projet.")
print("   - Construire un indice RAG: python -m src.rag_build")
print("   - Lancer l'UI: python ui/app.py")
print("   - Démo CLI: python demo_cli.py")

## 🎉 Installation terminée !

Votre environnement Agent Vocal IA est prêt. Vous pouvez maintenant :

1. **Construire des indices RAG** pour vos matières
2. **Tester les modules individuellement** (ASR, TTS, LLM)
3. **Lancer l'interface Gradio** pour une utilisation interactive
4. **Utiliser le mode CLI** pour des tests en ligne de commande

### Prochaines étapes

Consultez le `README.md` pour les instructions d'utilisation détaillées.